In [1]:
import os
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from transformers import AutoTokenizer, AutoModel
from imblearn.over_sampling import ADASYN
from xgboost import XGBClassifier

In [2]:
df = pd.read_excel('/content/Output.xlsx')

In [3]:
df

,Sequence_id,Sequence,Label
0,FP929040.1,GTACGTCGAAAAACGGCAGGCCGGGCAAGGCGAAGCCGCCACCCGG...,1
1,CP008823.1,CAAGAGGAAATTTAAAAAAAGTTTCTGACCGTGTTTTTTTTCACCA...,0
2,CP008897.1,CAAGAGGTAATTTAAAAAAAGTTTCTGACCGTGTTTTTTTTCACCA...,1
3,CP008905.1,CAAGAGGTAATTTAAAAAAAGTTTCTGACCGTGTTTTTTTTCACCA...,0
4,CP010377.1,CGGCCTAAATCGCGGGGCACTCGCGTCAGGATCGTTTACACTTGCC...,0
...,...,...,...
216,CP135475.1,GTGTCACTTTCGCTTTGGCAGCAGTGTCTTGCCCGATTGCAGGATG...,1
217,CP135494.1,GTGTCACTTTCGCTTTGGCAGCAGTGTCTTGCCCGATTGCAGGATG...,1
218,CP135498.1,GTGTCACTTTCGCTTTGGCAGCAGTGTCTTGCCCGATTGCAGGATG...,1
219,CP141537.1,AGAACTCCCTATAATGCGCCTCCATCGACACGGCGGATGTGAATCA...,1


## USING NT

In [4]:
model_name = "InstaDeepAI/nucleotide-transformer-2.5b-multi-species"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/101 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

...

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/278M [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of EsmModel were not initialized from the model checkpoint at InstaDeepAI/nucleotide-transformer-2.5b-multi-species and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


EsmModel(
  (embeddings): EsmEmbeddings(
    (word_embeddings): Embedding(4105, 2560, padding_idx=1)
    (dropout): Dropout(p=0.0, inplace=False)
    (position_embeddings): Embedding(1002, 2560, padding_idx=1)
  )
  (encoder): EsmEncoder(
    (layer): ModuleList(
      (0-31): 32 x EsmLayer(
        (attention): EsmAttention(
          (self): EsmSelfAttention(
            (query): Linear(in_features=2560, out_features=2560, bias=True)
            (key): Linear(in_features=2560, out_features=2560, bias=True)
            (value): Linear(in_features=2560, out_features=2560, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): EsmSelfOutput(
            (dense): Linear(in_features=2560, out_features=2560, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (LayerNorm): LayerNorm((2560,), eps=1e-12, elementwise_affine=True)
        )
        (intermediate): EsmIntermediate(
          (dense): Linear(in_features=25

In [5]:
df = df[['Sequence', 'Label']].dropna()
df['Label'] = df['Label'].astype(int)

In [6]:
# get embeddings
def get_embedding(seq):
    tokens = tokenizer(seq, return_tensors="pt", padding="max_length", truncation=True, max_length=512)
    tokens = {k: v.to(device) for k, v in tokens.items()}
    with torch.no_grad():
        outputs = model(**tokens)
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

embeddings, labels = [], []

for i, row in tqdm(df.iterrows(), total=len(df)):
    try:
        emb = get_embedding(row["Sequence"])
        embeddings.append(emb)
        labels.append(row["Label"])
    except Exception as e:
        print(f"⚠️ Error at index {i}: {e}")

X = np.array(embeddings)
y = np.array(labels)

100%|██████████| 221/221 [02:56<00:00,  1.25it/s]


In [7]:
adasyn = ADASYN(random_state=42)
X_resampled, y_resampled = adasyn.fit_resample(X, y)
print("✅ Dataset balanced using ADASYN.")

✅ Dataset balanced using ADASYN.


In [8]:
# === 5-Fold Stratified Cross-Validation with XGBoost ===
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accs, precs, recalls, f1s, aucs = [], [], [], [], []

for fold, (train_index, test_index) in enumerate(kf.split(X_resampled, y_resampled), 1):
    X_train, X_test = X_resampled[train_index], X_resampled[test_index]
    y_train, y_test = y_resampled[train_index], y_resampled[test_index]

    clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    y_proba = clf.predict_proba(X_test)

    accs.append(accuracy_score(y_test, y_pred))
    precs.append(precision_score(y_test, y_pred, average="weighted", zero_division=0))
    recalls.append(recall_score(y_test, y_pred, average="weighted", zero_division=0))
    f1s.append(f1_score(y_test, y_pred, average="weighted", zero_division=0))


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [04:32:23] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [04:32:32] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [04:32:41] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [04:32:48] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [04:32:56] WARNING: /w

In [9]:
    if len(np.unique(y_test)) == 2:
        auc = roc_auc_score(y_test, y_proba[:, 1])
    else:
        auc = roc_auc_score(y_test, y_proba, multi_class='ovr', average='weighted')
    aucs.append(auc)

    print(f"✅ Fold {fold} done.")

✅ Fold 5 done.


In [10]:
print("\n=== Cross-Validation Results (5-Fold, XGBoost, ADASYN) ===")
print(f"Accuracy:  {np.mean(accs):.4f} ± {np.std(accs):.4f}")
print(f"Precision: {np.mean(precs):.4f} ± {np.std(precs):.4f}")
print(f"Recall:    {np.mean(recalls):.4f} ± {np.std(recalls):.4f}")
print(f"F1 Score:  {np.mean(f1s):.4f} ± {np.std(f1s):.4f}")
print(f"ROC AUC:   {np.mean(aucs):.4f} ± {np.std(aucs):.4f}")


=== Cross-Validation Results (5-Fold, XGBoost, ADASYN) ===
Accuracy:  0.7406 ± 0.0430
Precision: 0.7462 ± 0.0433
Recall:    0.7406 ± 0.0430
F1 Score:  0.7379 ± 0.0439
ROC AUC:   0.8117 ± 0.0000


## USING DNA BERT

In [ ]:
def seq2kmer(seq: str, k: int = 6, overlap: bool = True) -> str:
    step = 1 if overlap else k
    return " ".join(seq[i:i+k] for i in range(0, len(seq)-k+1, step))

In [ ]:
print(seq2kmer("ATCGTACGATCG", k=6))

ATCGTA TCGTAC CGTACG GTACGA TACGAT ACGATC CGATCG


In [ ]:
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve
from datasets import Dataset


In [ ]:
df = pd.read_csv(r'/content/converted_output.csv').iloc[:, 1:]
df.columns = ['Sequence', 'Label']
df['text'] = df['Sequence'].str.upper().apply(lambda s: seq2kmer(s, k=6))

In [ ]:
model_id = "zhihan1996/DNA_bert_6"
tokenizer = BertTokenizerFast.from_pretrained(model_id, trust_remote_code=True)
max_length = 128

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

configuration_bert.py:   0%|          | 0.00/807 [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/zhihan1996/DNA_bert_6:
- configuration_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
def preprocess_for_fold(train_idx, test_idx):
    train, test = df.iloc[train_idx], df.iloc[test_idx]
    ds_tr = Dataset.from_dict({'text': train['text'], 'label': train['Label']})
    ds_te = Dataset.from_dict({'text': test['text'],  'label': test['Label']})

    def encode(batch):
        enc = tokenizer(
            batch["text"],
            truncation=True,
            padding="max_length",
            max_length=max_length,
            is_split_into_words=False
        )
        enc["labels"] = batch["label"]
        return enc

    ds_tr = ds_tr.map(encode, batched=True, remove_columns=ds_tr.column_names)
    ds_te = ds_te.map(encode, batched=True, remove_columns=ds_te.column_names)

    ds_tr.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
    ds_te.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
    return ds_tr, ds_te

In [ ]:
fold_metrics = {
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1': [],
    'roc_auc': []
}

for fold, (ti, vi) in enumerate(skf.split(df, df['Label']), 1):
    print(f"\n--- Fold {fold} ---")
    ds_tr, ds_te = preprocess_for_fold(ti, vi)
    model = BertForSequenceClassification.from_pretrained(model_id, num_labels=2)
    training_args = TrainingArguments(
        output_dir=f'./cv_results/fold{fold}',
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=32,
        learning_rate=2e-4,
        eval_strategy="epoch",
        logging_strategy="epoch",
        save_strategy="no",
        seed=42,
    )


--- Fold 1 ---


Map:   0%|          | 0/176 [00:00<?, ? examples/s]

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/359M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at zhihan1996/DNA_bert_6 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at zhihan1996/DNA_bert_6 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



--- Fold 5 ---


Map:   0%|          | 0/177 [00:00<?, ? examples/s]

Map:   0%|          | 0/44 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at zhihan1996/DNA_bert_6 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer = Trainer(model=model, args=training_args, train_dataset=ds_tr, eval_dataset=ds_te)
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tridippanja9 (tridippanja9-jis-institute-of-advanced-studies-research) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,0.690800,0.743764
2,0.652300,0.770651
3,0.652000,0.748543


TrainOutput(global_step=36, training_loss=0.6650524139404297, metrics={'train_runtime': 49.0506, 'train_samples_per_second': 10.826, 'train_steps_per_second': 0.734, 'total_flos': 34927992599040.0, 'train_loss': 0.6650524139404297, 'epoch': 3.0})

In [ ]:
    preds = trainer.predict(ds_te)
    y_true = preds.label_ids
    y_probs = torch.softmax(torch.tensor(preds.predictions), dim=1).numpy()
    y_pred = np.argmax(y_probs, axis=1)

    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_probs[:, 1])

In [ ]:
    print(f"Fold {fold} Accuracy : {acc:.4f}")
    print(f"Fold {fold} Precision: {prec:.4f}")
    print(f"Fold {fold} Recall   : {rec:.4f}")
    print(f"Fold {fold} F1-score : {f1:.4f}")
    print(f"Fold {fold} ROC AUC  : {roc_auc:.4f}")



Fold 5 Accuracy : 0.6136
Fold 5 Precision: 0.6136
Fold 5 Recall   : 1.0000
Fold 5 F1-score : 0.7606
Fold 5 ROC AUC  : 0.3976


In [ ]:
 fold_metrics['accuracy'].append(acc)
 fold_metrics['precision'].append(prec)
 fold_metrics['recall'].append(rec)
 fold_metrics['f1'].append(f1)
 fold_metrics['roc_auc'].append(roc_auc)

In [ ]:
print("\n=== Cross-validation Summary ===")
for metric, values in fold_metrics.items():
    print(f"{metric.capitalize()} - Mean: {np.mean(values):.4f} | Std: {np.std(values):.4f}")


=== Cross-validation Summary ===
Accuracy - Mean: 0.6136 | Std: 0.0000
Precision - Mean: 0.6136 | Std: 0.0000
Recall - Mean: 1.0000 | Std: 0.0000
F1 - Mean: 0.7606 | Std: 0.0000
Roc_auc - Mean: 0.3976 | Std: 0.0000
